In [434]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

# PyRQP (forward calculation)

## Input parameters

In [435]:
# Flows
riv_flow_mean = 100
riv_flow_sd = 93  # New code uses 5th percentile
dis_flow_mean = 20
dis_flow_sd = 8

# Water quality
riv_wq_mean = 2
riv_wq_sd = 1
dis_wq_mean = 15
dis_wq_sd = 7

# Correlations
corr_riv_dis_flow = 0.6
corr_riv_flow_wq = -0.3
corr_dis_flow_wq = -0.2

## Prepare all functionality

In [436]:
def transform_log_to_normal(lg_mean, lg_sd):
    """ """
    mean = np.log(lg_mean / ((1 + ((lg_sd**2) / (lg_mean**2))) ** 0.5))
    sd = (np.log(1 + (lg_sd**2) / (lg_mean**2))) ** 0.5
    return mean, sd

In [437]:
def calculate_covariance(corr, std_1, std_2):
    """
    This formula takes a correlation and two std
    and calculates the covariance matrix
    """
    cov = corr * std_1 * std_2
    return cov

## Calculate correlated lognormal random variable

In [438]:
def calculate_multivariate_log_normal(
    mean1, std1, mean2, std2, mean3, std3, mean4, std4, corr1_2, corr1_3, corr2_4
):
    """ """
    # Transform to normal
    mean1, std1 = transform_log_to_normal(mean1, std1)
    mean2, std2 = transform_log_to_normal(mean2, std2)
    mean3, std3 = transform_log_to_normal(mean3, std3)
    mean4, std4 = transform_log_to_normal(mean4, std4)

    # riv_flow, dis_flow, riv_qual, dis_qual
    # mean1, mean2, mean3, mean4 = 5, 10, 15, 20
    # std1, std2, std3, std4 = 5, 5, 5, 5
    # corr1_2, corr1_3, corr2_4 = 0.6, -0.3, -0.2

    cov1_2 = calculate_covariance(corr1_2, std1, std2)
    cov1_3 = calculate_covariance(corr1_3, std1, std3)
    cov2_4 = calculate_covariance(corr2_4, std2, std4)

    cov_matrix = [
        [std1**2, cov1_2, cov1_3, 0],
        [cov1_2, std2**2, 0, cov2_4],
        [cov1_3, 0, std3**2, 0],
        [0, cov2_4, 0, std4**2],
    ]

    cov_matrix = np.array(cov_matrix)

    data = np.random.multivariate_normal(
        [mean1, mean2, mean3, mean4], cov_matrix, size=10000
    )
    data = np.exp(data)

    df = pd.DataFrame(data, columns=["riv_flow", "dis_flow", "riv_qual", "dis_qual"])

    return df

## Calculate downstream quality column

In [439]:
df = calculate_multivariate_log_normal(
    # Flow
    riv_flow_mean,
    riv_flow_sd,
    dis_flow_mean ,
    dis_flow_sd,
    # Quality
    riv_wq_mean,
    riv_wq_sd,
    dis_wq_mean,
    dis_wq_sd,
    # Correlations
    corr_riv_dis_flow,
    corr_riv_flow_wq,
    corr_dis_flow_wq,
)

In [440]:
df = df.eval("downstream_flow = riv_flow + dis_flow")

df = df.eval(
    "downstream_qual = (riv_flow * riv_qual + dis_flow * dis_qual) / downstream_flow"
)

## Calculate descriptive statistics

In [441]:
stats = df.describe().T

In [442]:
stats["90pc"] = df.quantile(0.90)
stats["95pc"] = df.quantile(0.95)
stats["99pc"] = df.quantile(0.99)

In [443]:
stats

,count,mean,std,min,25%,50%,75%,max,90pc,95pc,99pc
riv_flow,10000.0,99.471974,88.285533,2.241762,43.025324,73.728045,125.344806,1015.086463,199.833314,268.165991,447.263540
dis_flow,10000.0,20.044423,8.038830,4.269176,14.328689,18.628638,24.084915,86.543312,30.524573,34.941902,45.764026
riv_qual,10000.0,2.004836,1.015138,0.295454,1.283172,1.784093,2.474776,9.690887,3.310226,3.922899,5.477047
dis_qual,10000.0,14.977906,6.963690,2.290223,10.078337,13.523052,18.246863,75.842965,23.996584,28.046700,37.931065
downstream_flow,10000.0,119.516397,92.918294,8.938622,59.882851,93.250218,148.701455,1052.526937,226.641822,295.219110,480.880230
downstream_qual,10000.0,4.743270,2.260721,0.914240,3.163564,4.277433,5.794991,23.885263,7.665408,9.023485,12.348298


In [444]:
df.corr()  # This is important information that should be part of the analysis

,riv_flow,dis_flow,riv_qual,dis_qual,downstream_flow,downstream_qual
riv_flow,1.000000,0.545891,-0.225042,0.006755,0.997369,-0.471200
dis_flow,0.545891,1.000000,0.009332,-0.179766,0.605189,-0.189423
riv_qual,-0.225042,0.009332,1.000000,0.002602,-0.213014,0.543505
dis_qual,0.006755,-0.179766,0.002602,1.000000,-0.009134,0.568119
downstream_flow,0.997369,0.605189,-0.213014,-0.009134,1.000000,-0.464094
downstream_qual,-0.471200,-0.189423,0.543505,0.568119,-0.464094,1.000000


# TODO

- Transform into a proper GitHub library
- Add extra functionality
    - Backward calculations
    - Decay (with optional decay rate and time or distance plus alpha/beta values)
    - Visualisation (e.g., histograms)